In [1]:
import math
import matplotlib.pyplot as plt
import keras
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
df=pd.read_csv("/content/BANKASIA.csv",parse_dates=['DATE'])
df.head()

,#,DATE,TRADING CODE,LTP*,HIGH,LOW,OPENP*,CLOSEP*,YCP,TRADE,VALUE (mn),VOLUME
0,439,2018-05-08,BANKASIA,17.1,17.5,16.8,17.5,16.8,17.5,158,6.782,"400,155"
1,438,2018-06-08,BANKASIA,16.9,17.3,16.8,16.8,16.9,16.8,160,11.494,"673,813"
2,437,2018-07-08,BANKASIA,17.2,17.4,16.9,17.3,17.2,16.9,218,16.415,"955,798"
3,436,2018-08-08,BANKASIA,17.4,17.4,16.9,17.1,17.3,17.2,242,16.895,"982,419"
4,435,2018-09-08,BANKASIA,17.3,17.8,17.0,17.5,17.1,17.3,336,14.668,"839,199"


In [2]:
# Sort DataFrame by date
df = df.sort_values('DATE')

# Double check the result
df.head()

,#,DATE,TRADING CODE,LTP*,HIGH,LOW,OPENP*,CLOSEP*,YCP,TRADE,VALUE (mn),VOLUME
36,403,2018-01-10,BANKASIA,16.7,17.0,16.6,16.7,16.8,16.8,57,2.271,"135,168"
59,380,2018-01-11,BANKASIA,17.1,17.8,17.1,17.6,17.2,17.4,134,4.479,"257,851"
37,402,2018-02-10,BANKASIA,16.7,16.9,16.7,16.7,16.7,16.8,52,1.806,"107,644"
79,360,2018-02-12,BANKASIA,17.5,17.5,17.3,17.3,17.4,17.3,37,1.489,"85,673"
16,423,2018-03-09,BANKASIA,17.4,17.6,17.1,17.5,17.2,17.5,262,11.665,"675,112"


In [3]:
df = df.set_index('DATE')
df.head(3)

,#,TRADING CODE,LTP*,HIGH,LOW,OPENP*,CLOSEP*,YCP,TRADE,VALUE (mn),VOLUME
DATE,,,,,,,,,,,
2018-01-10,403,BANKASIA,16.7,17.0,16.6,16.7,16.8,16.8,57,2.271,"135,168"
2018-01-11,380,BANKASIA,17.1,17.8,17.1,17.6,17.2,17.4,134,4.479,"257,851"
2018-02-10,402,BANKASIA,16.7,16.9,16.7,16.7,16.7,16.8,52,1.806,"107,644"


In [4]:
df['daily_ret'] = df['CLOSEP*'].pct_change()
df['daily_ret'] = df['daily_ret'].fillna(0.0)

In [5]:
df['volatility']= df['daily_ret'].rolling(window=21).std()
df['volatility'] = df['volatility'].fillna(0.0)

In [6]:
df['volatility'] = df['volatility']*100

In [7]:
import matplotlib.pyplot as plt 

In [8]:
df=df.reset_index()['volatility']

In [ ]:
plt.plot(df)

In [9]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))
df=scaler.fit_transform(np.array(df).reshape(-1,1))

In [ ]:
df.shape

In [10]:
##splitting dataset into train and test split where train=70 percent and test=30 percent
training_size=int(len(df)*0.70)
test_size=len(df)-training_size
train_data,test_data=df[0:training_size,:],df[training_size:len(df),:1]




In [ ]:

training_size,test_size

In [11]:
import numpy
# convert an array of values into a dataset matrix
def create_dataset(dataset, time_step=1):
	dataX, dataY = [],[]
	for i in range(len(dataset)-time_step-1):
		a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100 
		dataX.append(a)
		dataY.append(dataset[i + time_step, 0])
	return numpy.array(dataX), numpy.array(dataY)

In [12]:

# reshape into X=t,t+1,t+2,t+3 and Y=t+4
time_step = 100
X_train, y_train = create_dataset(train_data, time_step)
X_test, ytest = create_dataset(test_data, time_step)

In [13]:

print(X_train.shape), print(y_train.shape)

(206, 100)
(206,)


(None, None)

In [14]:
# reshape input to be [samples, time steps, features] which is required for LSTM
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

In [15]:
from numpy import array
from numpy import cumsum
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import Bidirectional
 

In [16]:

model=Sequential()
model.add(Bidirectional(LSTM(50,return_sequences=True,input_shape=(100,1))))
model.add(Bidirectional(LSTM(50,return_sequences=True)))
model.add(Bidirectional(LSTM(50)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

In [17]:

model.fit(X_train,y_train,validation_data=(X_test,ytest),epochs=100,batch_size=32,verbose=1)

Epoch 1/100
7/7 [==============================] - 16s 663ms/step - loss: 0.2569 - val_loss: 0.1459
Epoch 2/100
7/7 [==============================] - 2s 234ms/step - loss: 0.0283 - val_loss: 0.0377
Epoch 3/100
7/7 [==============================] - 2s 239ms/step - loss: 0.0241 - val_loss: 0.0735
Epoch 4/100
7/7 [==============================] - 2s 240ms/step - loss: 0.0165 - val_loss: 0.0374
Epoch 5/100
7/7 [==============================] - 2s 273ms/step - loss: 0.0159 - val_loss: 0.0395
Epoch 6/100
7/7 [==============================] - 2s 284ms/step - loss: 0.0115 - val_loss: 0.0398
Epoch 7/100
7/7 [==============================] - 2s 273ms/step - loss: 0.0114 - val_loss: 0.0345
Epoch 8/100
7/7 [==============================] - 2s 256ms/step - loss: 0.0101 - val_loss: 0.0363
Epoch 9/100
7/7 [==============================] - 2s 259ms/step - loss: 0.0098 - val_loss: 0.0308
Epoch 10/100
7/7 [==============================] - 2s 268ms/step - loss: 0.0075 - val_loss: 0.0309
Epoch 11

In [18]:
from google.colab import drive

In [19]:
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
filepath = "/content/drive/MyDrive/stock_price_BILSTM/modelsBANKASIA/model.h5"
model.save(filepath)


In [21]:
from keras.models import load_model
f1 = '/content/drive/MyDrive/stock_price_BILSTM/modelsBANKASIA/model.h5'
model = load_model(f1)

In [22]:
import tensorflow as tf
from sklearn.metrics import r2_score

In [23]:

### Lets Do the prediction and check performance metrics
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)

In [24]:

##Transformback to original form
train_predict = np.reshape(train_predict,(1, train_predict.size))
train_predict=scaler.inverse_transform(train_predict)
test_predict = np.reshape(test_predict,(1, test_predict.size))
test_predict=scaler.inverse_transform(test_predict)
X = np.reshape(ytest,(1, ytest.size))
Y_test =scaler.inverse_transform(X)

In [25]:
#Y_test = Y_test.transpose()
RMSE = math.sqrt(mean_squared_error(Y_test,test_predict))
RMSE

0.4956808672478345

In [26]:

from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(Y_test,test_predict)
print('MAE: %f' % mae)


MAE: 0.419117
